In [1]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import re

In [2]:
dbname = 'podcast'
username = 'lindsay'

In [3]:
# create connection to database
engine = create_engine('postgres://%s@localhost/%s'%(username, dbname))
print engine.url

postgres://lindsay@localhost/podcast


In [4]:
# create database
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [5]:
scrapeDf = pd.read_pickle('pkl/scraped_podcasts_COMPLETE.pkl')
apiDf = pd.read_pickle('pkl/itunes_podcasts_v3_COMPLETE.pkl')

In [6]:
# convert camelCase to camel_case
def convert(name):
    """Converts camelCase to camel_case."""
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

In [8]:
scrapeDf.columns = [convert(x) for x in scrapeDf.columns]
apiDf.columns = [convert(x) for x in apiDf.columns]

In [12]:
# join dfs
podcastDf = pd.merge(scrapeDf, apiDf, how='inner', on='collection_id')

In [15]:
# insert data into database
podcastDf.to_sql('podcasts', engine, if_exists='replace')

In [16]:
# connect for querying
con = None
con = psycopg2.connect(database = dbname, user = username)

In [17]:
# query
sql_query = """
SELECT * FROM podcasts WHERE collection_id=515836681;
"""

query_result = pd.read_sql_query(sql_query, con)
query_result

,index,also_subscribed,collection_id,episode_descriptions,episode_names,podcast_summary,artist_id,artist_name,artist_view_url,artwork_url100,...,track_count,track_explicitness,track_hd_price,track_hd_rental_price,track_id,track_name,track_price,track_rental_price,track_view_url,wrapper_type
0,0,"{508647267,544599736,435853369,515824283,28140...",515836681,"['In this episode of ""A Cast of Kings,"" Joanna...","['A Cast of Kings S1E10 - Fire and Blood', 'A ...","A quasi-weekly discussion of the plots, themes...",None,David Chen and Joanna Robinson,None,http://is1.mzstatic.com/image/thumb/Music/v4/0...,...,20,explicit,0,0,515836681,A Cast of Kings - A Game of Thrones Podcast,0,0,https://itunes.apple.com/us/podcast/cast-kings...,track
